<a href="https://colab.research.google.com/github/dmora4/network_measurements_course/blob/main/NML01_Ping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Intro
When we write here in CoLab, we are given an *instance* of a VM into Google's Cloud Network, thus everyone in this classroom is on a different VM. 

Some python libs are already installed on the VMs, but we can install more as we wish. 

Each time we close the notebook, everything is erased and resources are released and given to someone else [not persistent]. The maximum amount of time to keep notebook open is usually approximately 24 hrs... plenty of time.


Commands starting with `!` calls a terminal command on the VM

Try out the `ls` command:

In [ ]:
!ls

sample_data


`Sample_data` is a predefined folder in the VM used to store files

## Install useful libraries
`pythonping` is a Python implementation of `ping`

`scapy` is a Python library to manipulate packets




In [ ]:
!pip install pythonping
!pip install --pre scapy[basic]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.9 MB/s eta 0:00:00
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444349 sha256=8ca46d890d17fed12dfa3c49b02138532474becdf6e2833e9b3dc606a230cc04
  Stored in directory: /root/.cache/pip/wheels/98/ea/08/164e840ab2c83b892bf8b193ce9d92d029dc3e5f2b75319953
Successfully built scapy


## Find IP of the VM we are using in Google's Cloud Network

`json` is used to parse the response

`urllib` is to to send requests to a Web Service that gives provides a user's IP addr. In our case, using this Web Service from the VM will give us the IP of the VM


In [ ]:
import urllib.request 
import json

ip = urllib.request.urlopen('https://api.ipify.org').read().decode('UTF-8')
ip_info = json.loads(urllib.request.urlopen('http://ip-api.com/json/' + ip).read())

print('External IP\t', ip)
print('Organizaiton:\t', ip_info["org"])
print('Locaiton:\t', ip_info["city"], ',',ip_info["country"])

External IP	 34.145.203.132
Organizaiton:	 Google Cloud (us-east4)
Locaiton:	 Washington , United States


Now let's use the Python implementation of `ping`

We are going to ping a relatively faraway server... off to New Zealand!

In [ ]:
from pythonping import ping
target = 'ftp.nz.debian.org'
ping(target, verbose=False, count=20)


Reply from 163.7.134.112, 29 bytes in 213.98ms
Reply from 163.7.134.112, 29 bytes in 213.03ms
Reply from 163.7.134.112, 29 bytes in 213.2ms
Reply from 163.7.134.112, 29 bytes in 213.12ms
Reply from 163.7.134.112, 29 bytes in 213.15ms
Reply from 163.7.134.112, 29 bytes in 213.05ms
Reply from 163.7.134.112, 29 bytes in 213.13ms
Reply from 163.7.134.112, 29 bytes in 213.19ms
Reply from 163.7.134.112, 29 bytes in 213.12ms
Reply from 163.7.134.112, 29 bytes in 213.12ms
Reply from 163.7.134.112, 29 bytes in 213.14ms
Reply from 163.7.134.112, 29 bytes in 213.17ms
Reply from 163.7.134.112, 29 bytes in 213.14ms
Reply from 163.7.134.112, 29 bytes in 213.08ms
Reply from 163.7.134.112, 29 bytes in 213.08ms
Reply from 163.7.134.112, 29 bytes in 213.06ms
Reply from 163.7.134.112, 29 bytes in 213.15ms
Reply from 163.7.134.112, 29 bytes in 213.18ms
Reply from 163.7.134.112, 29 bytes in 213.23ms
Reply from 163.7.134.112, 29 bytes in 213.21ms

Round Trip Times min/avg/max is 213.03/213.18/213.98 ms

Now we introduce some Python libaries handy for processing packets. 

Using `scapy` we will implement our own versions of:
*   `scapy`
*   `traceout`

## Intro to `scapy`

`scapy` allows us to *create* network packets by easily picking several pre-existing configurations.

For example, let's see the header of the default IP packet provded by `scapy`:

In [ ]:
from scapy.all import *

IP().show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = 0
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \



We can do the same for other protocols (ICMP, TCP, UDP, etc...):

In [ ]:
ICMP().show()


TCP().show()

UDP().show()

###[ ICMP ]### 
  type      = echo-request
  code      = 0
  chksum    = None
  id        = 0x0
  seq       = 0x0
  unused    = ''

###[ TCP ]### 
  sport     = 20
  dport     = 80
  seq       = 0
  ack       = 0
  dataofs   = None
  reserved  = 0
  flags     = S
  window    = 8192
  chksum    = None
  urgptr    = 0
  options   = ''

###[ UDP ]### 
  sport     = 53
  dport     = 53
  len       = None
  chksum    = None




Consider the TCP header. Recall the standard HTTP port is 80. `scapy` sets the port (`dport` field) to 80 in its default configuration of a TCP packet. 

Fields of the header are fields of the object and are to be changed inside the `(field=new_val, ...)`. The division operator `\` is overloaded in `scapy` to stack packets. See below we:
*   destination fiel `dst` in the IP packet header, and
*   stack an `ICMP` packet inside and `IP` packet

Note, we previously created a variable names `target` set host name of this New Zealand server, but the hostname reused here for clarity. 

In [ ]:
packet = IP(dst= 'ftp.nz.debian.org') / ICMP()
packet.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = 1
  chksum    = None
  src       = 172.28.0.12
  dst       = Net("ftp.nz.debian.org/32")
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x0
     seq       = 0x0
     unused    = ''



Use `sr()`, which stands for *send* and *recieve*, to send the packet we previously created. This `sr()` returns two Python lists: 
*   the list of *answered* packets, and
*   the list of *unanswered* packets

Recall, this packet is an `echo-request` message (by default) with the New Zealand server as its destination.

In [ ]:
ans, unans = sr(packet, verbose=False)
print('Received:\n' , len(ans), 'packet/s\n')
print('Summary:')
#print('---------------- request ---------------- \t\t ---------------- response ----------------')
#print('[headers] \t[src] > [dst]\t\t[type]\t[seq] ==> [headers]\t[src] > [dst]\t[type] [seq]')
print(ans.summary())

Received:
 1 packet/s

Summary:
IP / ICMP 172.28.0.12 > 163.7.134.112 echo-request 0 ==> IP / ICMP 163.7.134.112 > 172.28.0.12 echo-reply 0
None


Observe the packet we sent was of type `echo-request`, thus in the response packet, an `echo-reply` is appropriate. 

In [ ]:
# Let's use query and answer variables to easily and clearly access the packets 
# involved in this sr exchange
query = ans[0][0]
answer = ans[0][1]

print(query.summary())
print(answer.summary())

IP / ICMP 172.28.0.12 > 163.7.134.112 echo-request 0
IP / ICMP 163.7.134.112 > 172.28.0.12 echo-reply 0


This is proof that `query` and `answer` are the same as `ans[0][0]` and `ans[0][1]`

In [ ]:
# .sent_time is the time the query packet was SENT. Note, in units of ns
print(query.sent_time, 'ns')

# .time is the time the answer packet was RECIEVED. Note, in units of ns
print(answer.time, 'ns')

rtt = 1000* (answer.time-query.sent_time)
print('%.3f' % rtt, 'ms') 
#values are given in ns, multiply by 1000 to get in ms

1677185225.2248254 ns
1677185225.4400218 ns
215.196 ms


### Remark on private vesus external IP addresses

This Google farm is a LAN, thus this VM has *private* address. When encountered with sending messages in/out the LAN, a border router *magically* tranforms private addresses to external addresses (and vice versa). When we send a message and hardcode our src address with the external IP, the response message will try to reply to the external IP, and the border router will just keep the message for itself. This response message will only reach the border router address... it will not reach our specific VM machine. In order to reach our specific VM machine, we should keep our private address as the src so that when the border address recieves the reply message, it knows to translate the external message back to our private address and the reply is reach our specfic VM! 

## Implement our own version of `ping`

In [ ]:
# Report min, max, avg rtt (exactly like ping)...
# Our version does not print std-dev... :(
def ping2(dest, count, ttl=64, verbose=False):
  rcv_count = 0
  sent_count = 0
  lost_count = 0
  rtt_sum = 0.0
  rtt_min = 1000000.0
  rtt_max = 0.0
  
  for i in range(count):
    packet = IP(dst = dest, ttl=ttl) / ICMP(seq = i)
    ans, unans = sr(packet, verbose=False)
    sent_count = sent_count + 1
    

    if(len(ans) > 0):
      rtt = 1000*(ans[0][1].time - ans[0][0].sent_time)
      if(verbose == True):
        print(len(ans[0][1]),'bytes from', ans[0][1].src, ': icmp_seq=', ans[0][1].seq,'ttl=', packet.ttl, 'time=', '%.3f'% rtt, 'ms')
      rcv_count = rcv_count + 1
      rtt_sum = rtt_sum + rtt
      if(len(unans)>0):
        lost = lost + len(unans)
      if(rtt > rtt_max):
        rtt_max = rtt
      if(rtt < rtt_min):
        rtt_min = rtt    
  
  rtt_avg = rtt_sum / rcv_count

  print('--- ping statistics ---')
  loss_rate = 100.0*(lost_count)/sent_count
  print(sent_count, 'packets transmitted, ', rcv_count, 'packets received, ', loss_rate, '% packet loss')
  print('round-trip min/avg/max= ', '%.3f'% rtt_min,'/', '%.3f'% rtt_avg,'/', '%.3f'% rtt_max)



ping2(target, 20, verbose=True)

#question: do we get the size of the mssg... is it just length(ans[0][1]) ?


28 bytes from 163.7.134.112 : icmp_seq= 0 ttl= 64 time= 214.220 ms
28 bytes from 163.7.134.112 : icmp_seq= 1 ttl= 64 time= 213.414 ms
28 bytes from 163.7.134.112 : icmp_seq= 2 ttl= 64 time= 213.604 ms
28 bytes from 163.7.134.112 : icmp_seq= 3 ttl= 64 time= 213.482 ms
28 bytes from 163.7.134.112 : icmp_seq= 4 ttl= 64 time= 213.407 ms
28 bytes from 163.7.134.112 : icmp_seq= 5 ttl= 64 time= 213.356 ms
28 bytes from 163.7.134.112 : icmp_seq= 6 ttl= 64 time= 213.481 ms
28 bytes from 163.7.134.112 : icmp_seq= 7 ttl= 64 time= 213.688 ms
28 bytes from 163.7.134.112 : icmp_seq= 8 ttl= 64 time= 213.566 ms
28 bytes from 163.7.134.112 : icmp_seq= 9 ttl= 64 time= 213.506 ms
28 bytes from 163.7.134.112 : icmp_seq= 10 ttl= 64 time= 213.359 ms
28 bytes from 163.7.134.112 : icmp_seq= 11 ttl= 64 time= 213.458 ms
28 bytes from 163.7.134.112 : icmp_seq= 12 ttl= 64 time= 213.407 ms
28 bytes from 163.7.134.112 : icmp_seq= 13 ttl= 64 time= 213.461 ms
28 bytes from 163.7.134.112 : icmp_seq= 14 ttl= 64 time= 2

In [ ]:
!apt-get install traceroute

Reading package lists... Done
Building dependency tree       
Reading state information... Done
traceroute is already the newest version (1:2.1.0-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
traceroute to ftp.nz.debian.org (163.7.134.112), 30 hops max, 60 byte packets
 1  172.28.0.1 (172.28.0.1)  0.057 ms  0.007 ms  0.006 ms
 2  66.249.94.186 (66.249.94.186)  69.321 ms  69.471 ms  69.467 ms
 3  six.reannz.co.nz (206.81.81.203)  69.458 ms  69.455 ms  69.450 ms
 4  and33-hnl.reannz.co.nz (210.7.33.244)  117.890 ms  117.888 ms  117.885 ms
 5  and31-mgw.reannz.co.nz (210.7.33.246)  211.949 ms  211.947 ms  211.944 ms
 6  and12-nsh.reannz.co.nz (210.7.33.242)  211.957 ms  211.203 ms  211.182 ms
 7  163.7.134.120 (163.7.134.120)  213.470 ms  213.179 ms  213.387 ms
 8  163.7.134.112 (163.7.134.112)  213.384 ms  213.380 ms  213.394 ms


In [ ]:
!traceroute -I ftp.nz.debian.org

traceroute to ftp.nz.debian.org (163.7.134.112), 30 hops max, 60 byte packets
 1  172.28.0.1 (172.28.0.1)  0.141 ms  0.015 ms  0.006 ms
 2  66.249.94.186 (66.249.94.186)  70.064 ms  70.047 ms  70.150 ms
 3  six.reannz.co.nz (206.81.81.203)  70.065 ms  70.061 ms  70.255 ms
 4  and33-hnl.reannz.co.nz (210.7.33.244)  117.731 ms  117.728 ms  117.724 ms
 5  and31-mgw.reannz.co.nz (210.7.33.246)  212.842 ms  212.838 ms  212.830 ms
 6  and12-nsh.reannz.co.nz (210.7.33.242)  215.656 ms  212.319 ms  212.294 ms
 7  163.7.134.120 (163.7.134.120)  214.531 ms  214.338 ms  214.319 ms
 8  163.7.134.112 (163.7.134.112)  214.288 ms  214.285 ms  214.281 ms


In [ ]:
import socket 
def traceroute2(hostname, maxTTL):

  hop = 0
  hostip = socket.gethostbyname(hostname)

  print('traceroute2 to ', hostname, '(', hostip, ')', maxTTL, 'hops max, ', '##? byte packets')
  
  msgtype = 11

  while(hop < maxTTL and msgtype != 0):
    ttl1 = IP(dst = hostname, ttl= hop + 1) / ICMP(seq=1)

    ans1, unans1 = sr(ttl1, verbose=False, timeout=3)

    interim_hostip = ans1[0][1].src
    interim_hostname = interim_hostip

    try: # sometimes the hostname is resolvable, sometimes it is not
      interim_hostname = socket.gethostbyaddr(interim_hostip)[0]
    
    except:     
      interim_hostname = interim_hostip

    msgtype = ans1[0][1].type

    if(len(ans1)>0): # we got a response:
      rtt1 = 1000*(ans1[0][1].time - ans1[0][0].sent_time)

    print(hop + 1, interim_hostname,'(',interim_hostip,')', '%.3f' % rtt1,'ms')
    hop = hop + 1

traceroute2(target, 20)




traceroute2 to  ftp.nz.debian.org ( 163.7.134.112 ) 20 hops max,  ##? byte packets
1 172.28.0.1 ( 172.28.0.1 ) 0.076 ms
2 66.249.94.186 ( 66.249.94.186 ) 68.856 ms
3 six.reannz.co.nz ( 206.81.81.203 ) 69.472 ms
4 and33-hnl.reannz.co.nz ( 210.7.33.244 ) 118.187 ms
5 and31-mgw.reannz.co.nz ( 210.7.33.246 ) 212.491 ms
6 and12-nsh.reannz.co.nz ( 210.7.33.242 ) 212.952 ms
7 163.7.134.120 ( 163.7.134.120 ) 214.909 ms
8 163.7.134.112 ( 163.7.134.112 ) 214.559 ms
